# Production Forecasting POC (Oil & Gas)

This notebook downloads monthly production-by-field data from a configurable internet source (CSV). If not available, it generates a realistic synthetic dataset. It then performs EDA and builds a SARIMAX forecasting model with walk-forward backtesting and plots.

Run all cells top-to-bottom.


In [ ]:
# Imports and setup
import os
import io
import sys
import math
import json
import time
import warnings
from dataclasses import dataclass
from typing import Optional, List

import numpy as np
import pandas as pd
import requests
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_absolute_error, mean_squared_error

warnings.filterwarnings("ignore")
plt.style.use("seaborn-v0_8-whitegrid")

DATA_URL = os.environ.get("PROD_DATA_URL", "")  # optional CSV URL
DEFAULT_FIELDS = ["FieldA", "FieldB"]  # used for synthetic fallback

@dataclass
class Config:
    target_column: str = "oil"
    date_column: str = "date"
    field_column: str = "field"
    frequency: str = "MS"  # month start

CFG = Config()



In [ ]:
# Data loading utilities

def try_download_csv(url: str, timeout: int = 30) -> Optional[pd.DataFrame]:
    if not url:
        return None
    try:
        r = requests.get(url, timeout=timeout)
        r.raise_for_status()
        df = pd.read_csv(io.StringIO(r.text))
        return df
    except Exception as e:
        print(f"Download failed from {url}: {e}")
        return None


def make_synthetic(start="2015-01-01", periods=120, fields=None) -> pd.DataFrame:
    if fields is None:
        fields = DEFAULT_FIELDS
    idx = pd.date_range(start=start, periods=periods, freq=CFG.frequency)
    rows = []
    rng = np.random.default_rng(42)
    for field in fields:
        base = rng.uniform(5000, 15000)
        decline = rng.uniform(0.002, 0.01)  # monthly decline
        season_amp = rng.uniform(0.02, 0.08)
        noise_sd = rng.uniform(50, 200)
        series = []
        for t in range(periods):
            seasonal = 1 + season_amp * math.sin(2 * math.pi * t / 12.0)
            value = base * ((1 - decline) ** t) * seasonal + rng.normal(0, noise_sd)
            series.append(max(value, 0))
        for dt, val in zip(idx, series):
            rows.append({CFG.date_column: dt, CFG.field_column: field, CFG.target_column: val})
    df = pd.DataFrame(rows)
    return df


def normalize_columns(df: pd.DataFrame) -> pd.DataFrame:
    cols = {c.lower().strip(): c for c in df.columns}
    # try map common variants
    mapping = {}
    for lc, orig in cols.items():
        if lc in {"date", "month", "period"}:
            mapping[orig] = CFG.date_column
        elif lc in {"field", "asset", "area"}:
            mapping[orig] = CFG.field_column
        elif lc in {"oil", "oil_bbl", "oil_production", "production", "value"}:
            mapping[orig] = CFG.target_column
    df2 = df.rename(columns=mapping)
    return df2


def coerce_types(df: pd.DataFrame) -> pd.DataFrame:
    if CFG.date_column in df.columns:
        df[CFG.date_column] = pd.to_datetime(df[CFG.date_column])
        df = df.sort_values(CFG.date_column)
    if CFG.field_column in df.columns:
        df[CFG.field_column] = df[CFG.field_column].astype(str)
    if CFG.target_column in df.columns:
        df[CFG.target_column] = pd.to_numeric(df[CFG.target_column], errors="coerce")
    return df


def load_data(data_url: str) -> pd.DataFrame:
    df = try_download_csv(data_url)
    if df is None:
        print("Using synthetic fallback dataset.")
        df = make_synthetic()
    df = normalize_columns(df)
    df = coerce_types(df)
    # basic sanity filtering
    df = df.dropna(subset=[CFG.date_column, CFG.field_column, CFG.target_column])
    return df


df_raw = load_data(DATA_URL)
df_raw.head()


In [ ]:
# EDA
print(df_raw.describe(include="all").T.head(10))

fig, ax = plt.subplots(figsize=(10,4))
for name, g in df_raw.groupby(CFG.field_column):
    g = g.sort_values(CFG.date_column)
    ax.plot(g[CFG.date_column], g[CFG.target_column], label=name)
ax.set_title("Monthly production by field")
ax.set_xlabel("Date")
ax.set_ylabel(CFG.target_column)
ax.legend(loc="upper right")
plt.show()

# pivot to wide for modeling per field
fields = sorted(df_raw[CFG.field_column].unique())
print(f"Fields: {fields}")


In [ ]:
# Walk-forward SARIMAX baseline per field

def walk_forward_sarimax(series: pd.Series, order=(1,1,1), seasonal_order=(1,1,1,12), train_ratio=0.8):
    series = series.asfreq(CFG.frequency)
    n = len(series)
    split = int(n * train_ratio)
    history = series.iloc[:split]
    test = series.iloc[split:]
    preds = []
    for t in range(len(test)):
        model = SARIMAX(history, order=order, seasonal_order=seasonal_order, enforce_stationarity=False, enforce_invertibility=False)
        res = model.fit(disp=False)
        fc = res.forecast(steps=1)
        preds.append(float(fc.iloc[0]))
        history = pd.concat([history, test.iloc[t:t+1]])
    idx = test.index
    y_true = test.values
    y_pred = np.array(preds)
    mae = mean_absolute_error(y_true, y_pred)
    rmse = math.sqrt(mean_squared_error(y_true, y_pred))
    return idx, y_true, y_pred, mae, rmse

metrics = []
plots = []
for field in fields:
    s = (df_raw[df_raw[CFG.field_column]==field]
            .set_index(CFG.date_column)[CFG.target_column]
            .asfreq(CFG.frequency)
            .interpolate(limit_direction="both"))
    idx, y_true, y_pred, mae, rmse = walk_forward_sarimax(s)
    metrics.append({"field": field, "mae": mae, "rmse": rmse})
    # Plot
    fig, ax = plt.subplots(figsize=(10,4))
    ax.plot(s.index, s.values, label="actual", alpha=0.6)
    ax.plot(idx, y_pred, label="pred (walk-forward)")
    ax.set_title(f"Forecast vs Actual - {field}")
    ax.set_xlabel("Date")
    ax.set_ylabel(CFG.target_column)
    ax.legend()
    plt.show()

pd.DataFrame(metrics).sort_values("rmse")


In [ ]:
# Simple explainability: recent change features and correlations

df_features = []
for field in fields:
    g = df_raw[df_raw[CFG.field_column]==field].sort_values(CFG.date_column).copy()
    g["lag1"] = g[CFG.target_column].shift(1)
    g["chg1"] = g[CFG.target_column].diff(1)
    g["ma3"] = g[CFG.target_column].rolling(3).mean()
    g["ma6"] = g[CFG.target_column].rolling(6).mean()
    g["y"] = g[CFG.target_column]
    g["field"] = field
    df_features.append(g)
X = pd.concat(df_features, ignore_index=True).dropna()

corr = X[["y","lag1","chg1","ma3","ma6"]].corr()
print(corr)

sns.pairplot(X[["y","lag1","chg1","ma3","ma6"]].sample(min(2000, len(X))), corner=True)
plt.suptitle("Feature relationships", y=1.02)
plt.show()
